In [1]:
dbutils.widgets.text("name", "")
dbutils.widgets.text("bucket_name", "")
dbutils.widgets.text("path", "")
dbutils.widgets.text("delimiter", "")
dbutils.widgets.text("extension", "")

In [2]:
name = dbutils.widgets.get("name")
bucket_name = dbutils.widgets.get("bucket_name")
path = dbutils.widgets.get("path")
delimiter = dbutils.widgets.get("delimiter")
extension = dbutils.widgets.get("extension")

In [3]:
%run ../Datasets/DatasetBase

In [4]:
%run ../Utils/Utils

In [5]:
bucket_write = dbutils.secrets.get(scope = "aws", key = "aws-bucket-name")
DatasetBase.createBucket(bucket_write)
count = 1

complete_path = f'/mnt/{bucket_write}/raw/datasets'

from datetime import datetime
from pyspark.sql.functions import col, max

if file_exists(complete_path):
    oldDataFrame = DatasetBase.loadDatabase(bucket_write, complete_path)
    
    if oldDataFrame.filter(col('name') == name).count() < 1:
        count = oldDataFrame.agg(max(col('id'))).collect()[0][0] + 1  
        conf = [(count, name, bucket_name, path, delimiter, extension, datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"))]
        newDataFrame = spark.createDataFrame(conf, ['id', 'name', 'bucket_name', 'path', 'delimiter', 'extension', 'created_at'])
        df = newDataFrame.union(oldDataFrame)
    else:
        dbutils.notebook.exit('Dataset já cadastrado!!')
else:
    count = 1
    conf = [(count, name, bucket_name, path, delimiter, extension, datetime.now().strftime("%Y-%m-%dT%H:%M:%S.000Z"))]
    df = spark.createDataFrame(conf, ['id', 'name', 'bucket_name', 'path', 'delimiter', 'extension', 'created_at'])

try:
    DatasetBase.writeDataset(df, bucket_write, complete_path)
    print('Dataset cadastrado com Sucesso!!!')
except Exception:
    dbutils.notebook.exit('ERROR: Erro na escrita do Dataset')

Dataset cadastrado com Sucesso!!!

In [6]:
df = DatasetBase.loadDatabase(bucket_name, complete_path)
display(df)

id,name,bucket_name,path,delimiter,extension,created_at
2,OrderStatuses,ifood-data-architect-test-source,status.json.gz,,json,2020-08-03T05:32:48.000Z
3,Consumer,ifood-data-architect-test-source,consumer.csv.gz,",",csv,2020-08-03T05:33:27.000Z
4,Restaurant,ifood-data-architect-test-source,restaurant.csv.gz,",",csv,2020-08-03T05:33:55.000Z
1,Order,ifood-data-architect-test-source,order.json.gz,,json,2020-08-03T05:32:10.000Z
